# Checking the functions of the pipeline

Imports

In [1]:
from src.unify_format import unify_data_format
from src.variable_matching import match_variables
from src.temporal_resolution import consolidate_time_resolution
from src.match import aggregate_and_merge_files

import os
import xarray as xr

import src.config

Downloading data

In [2]:
file_path_MetO = "/Users/bean/Documents/Doctorate/Fall_24/Mokbel8735_Adv_Databases/climate_data_integration/data/raw/HadISST.2.2.0.0_sea_ice_concentration.nc"
met = xr.open_dataset(file_path_MetO)

file_path_NOAA = "/Users/bean/Documents/Doctorate/Fall_24/Mokbel8735_Adv_Databases/climate_data_integration/data/raw/icec.mon.mean.nc"
noa = xr.open_dataset(file_path_NOAA)

### Standardizing variable names
**match_variables**

Current file column names:

In [49]:
met.dims.keys()

/var/folders/mr/2yqg72xx2l3bscg4fdjn78_c0000gn/T/ipykernel_5304/2489301570.py:1: FutureWarning: The return type of `Dataset.dims` will be changed to return a set of dimension names in future, in order to be more consistent with `DataArray.dims`. To access a mapping from dimension names to lengths, please use `Dataset.sizes`.
  met.dims.keys()


KeysView(FrozenMappingWarningOnValuesAccess({'time': 2048, 'nv': 2, 'latitude': 180, 'longitude': 360}))

In [58]:
met.data_vars.keys()

KeysView(Data variables:
    time_bnds  (time, nv) float32 16kB ...
    sic        (time, latitude, longitude) float32 531MB ...)

In [57]:
noa.dims.keys()

/var/folders/mr/2yqg72xx2l3bscg4fdjn78_c0000gn/T/ipykernel_5304/2353949293.py:1: FutureWarning: The return type of `Dataset.dims` will be changed to return a set of dimension names in future, in order to be more consistent with `DataArray.dims`. To access a mapping from dimension names to lengths, please use `Dataset.sizes`.
  noa.dims.keys()


KeysView(FrozenMappingWarningOnValuesAccess({'lat': 180, 'lon': 360, 'time': 2040}))

In [59]:
noa.data_vars.keys()

KeysView(Data variables:
    icec     (time, lat, lon) float32 529MB ...)

In [3]:
st = match_variables(input_data=[file_path_MetO, file_path_NOAA], standard_col_names=src.config.standard_col_names)

/Users/bean/Documents/Doctorate/Fall_24/Mokbel8735_Adv_Databases/climate_data_integration/src/variable_matching.py:27: FutureWarning: The return type of `Dataset.dims` will be changed to return a set of dimension names in future, in order to be more consistent with `DataArray.dims`. To access a mapping from dimension names to lengths, please use `Dataset.sizes`.
  for current_name in data.dims.keys():  # Check dimension names
/Users/bean/Documents/Doctorate/Fall_24/Mokbel8735_Adv_Databases/climate_data_integration/src/variable_matching.py:27: FutureWarning: The return type of `Dataset.dims` will be changed to return a set of dimension names in future, in order to be more consistent with `DataArray.dims`. To access a mapping from dimension names to lengths, please use `Dataset.sizes`.
  for current_name in data.dims.keys():  # Check dimension names


In [4]:
new_met = xr.open_dataset(st[0])
print(new_met.dims.keys())
print(new_met.data_vars.keys())
new_noa = xr.open_dataset(st[1])
print(new_noa.dims.keys())
print(new_noa.data_vars.keys())

KeysView(FrozenMappingWarningOnValuesAccess({'datetime': 2048, 'sea_ice_temp': 2, 'latitude': 180, 'longitude': 360}))
KeysView(Data variables:
    time_bnds  (datetime, sea_ice_temp) float32 16kB ...)
KeysView(FrozenMappingWarningOnValuesAccess({'latitude': 180, 'longitude': 360, 'datetime': 2040}))
KeysView(Data variables:
    sea_ice_temp  (datetime, latitude, longitude) float32 529MB ...)


/var/folders/mr/2yqg72xx2l3bscg4fdjn78_c0000gn/T/ipykernel_9825/1962799246.py:2: FutureWarning: The return type of `Dataset.dims` will be changed to return a set of dimension names in future, in order to be more consistent with `DataArray.dims`. To access a mapping from dimension names to lengths, please use `Dataset.sizes`.
  print(new_met.dims.keys())
/var/folders/mr/2yqg72xx2l3bscg4fdjn78_c0000gn/T/ipykernel_9825/1962799246.py:5: FutureWarning: The return type of `Dataset.dims` will be changed to return a set of dimension names in future, in order to be more consistent with `DataArray.dims`. To access a mapping from dimension names to lengths, please use `Dataset.sizes`.
  print(new_noa.dims.keys())


### Get same file format and check the grid map information
**unify_data_format**

In [5]:
u_data_0, g_0 = unify_data_format(st[0])
u_data_1, g_1 = unify_data_format(st[1])

In [6]:
print(u_data_1)

/Users/bean/Documents/Doctorate/Fall_24/Mokbel8735_Adv_Databases/climate_data_integration/data/processed/st_cols_icec.mon.mean.nc


In [7]:
print(g_1)

No CRS or grid mapping information found in the file.


### Resample datasets into monthly resolution
**consolidate_time_resolution**

In [6]:
m_0 = consolidate_time_resolution(u_data_0)
m_1 = consolidate_time_resolution(u_data_1)

	Temporal resolution: M
	Temporal resolution: M


In [9]:
ds_0 = xr.open_dataset(m_0)
ds_0

<xarray.Dataset> Size: 35kB
Dimensions:    (datetime: 2048, sea_ice_temp: 2, latitude: 180, longitude: 360)
Coordinates:
  * latitude   (latitude) float32 720B 89.5 88.5 87.5 86.5 ... -87.5 -88.5 -89.5
  * longitude  (longitude) float32 1kB 0.5 1.5 2.5 3.5 ... 357.5 358.5 359.5
  * datetime   (datetime) datetime64[ns] 16kB 1850-01-31 ... 2020-08-31
Dimensions without coordinates: sea_ice_temp
Data variables:
    time_bnds  (datetime, sea_ice_temp) float32 16kB ...
Attributes:
    Title:                      Monthly 1 degree version of HadISST.2.2.0.0
    source:                     Met Office
    reference:                  update of Titchner and Rayner 2015 DOI: 10.10...
    Conventions:                CF-1.0
    history:                    5/10/2020 converted to netcdf
    supplementary_information:  contact holly.titchner@metoffice.gov.uk or jo...

In [10]:
ds_1 = xr.open_dataset(m_1)
ds_1

<xarray.Dataset> Size: 529MB
Dimensions:       (datetime: 2040, latitude: 180, longitude: 360)
Coordinates:
  * latitude      (latitude) float32 720B 89.5 88.5 87.5 ... -87.5 -88.5 -89.5
  * longitude     (longitude) float32 1kB 0.5 1.5 2.5 3.5 ... 357.5 358.5 359.5
  * datetime      (datetime) datetime64[ns] 16kB 1850-01-31 ... 2019-12-31
Data variables:
    sea_ice_temp  (datetime, latitude, longitude) float32 529MB ...
Attributes:
    title:            created 05/2014 from data provided by JRA
    history:          Created 12/2012 from data obtained from JRA by ESRL/PSD
    platform:         Analyses
    citation:         Hirahara, S., Ishii, M., and Y. Fukuda,2014: Centennial...
    institution:      NOAA ESRL/PSD
    Conventions:      CF-1.2
    dataset_title:    COBE-SST2 Sea Surface Temperature and Ice
    original_source:  https://climate.mri-jma.go.jp/pub/ocean/cobe-sst2/
    References:       https://www.psl.noaa.gov/data/gridded/cobe2.html

### Aggregating and merging files
**aggregate_and_merge_files**